In [108]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [109]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [110]:
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS = 10000
LOSS_FUNCTION = nn.MSELoss()

In [111]:
def data_workflow(input_data, target_value):
    X = input_data.drop([target_value], axis=1)
    y = input_data[target_value]

    #print(X)
    #print(y)

    X = X.to_numpy()
    y = y.to_numpy()

    X = torch.from_numpy(X)
    y = torch.from_numpy(y)

    data = TensorDataset(X, y)

    train_ds, test_ds = train_test_split(data, test_size=0.2, random_state=25)

    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True)

    print(f"training data: {train_dl}\n test data: {test_dl}")
    return train_dl, test_dl
    



In [112]:
csv_filepath = "./final_harvest_data.csv"

data = pd.read_csv(csv_filepath)
DATA = data.drop(['date', 'index', 'plant_id', 'tray_id', 'row', 'column'], axis=1)

DATA

,LFW_g,LDW_g,LA_mm2,length_mm,width_mm,height_mm,plant_area,plant_convex_hull_area,plant_solidity,plant_perimeter,plant_width,plant_height,plant_longest_path,plant_convex_hull_vertices,plant_ellipse_major_axis,plant_ellipse_minor_axis,plant_ellipse_angle,plant_ellipse_eccentricity
0,1.30,0.078,31.95,4.3,5.2,5.8,1211,1456.0,0.831731,189.923880,49,42,346,16,45.684494,40.965988,92.878510,0.442608
1,2.10,0.148,44.10,5.3,5.7,5.5,1412,1556.5,0.907164,181.338094,56,41,370,20,53.368065,37.484673,96.255425,0.711802
2,3.36,0.196,67.61,7.3,6.5,8.9,1303,1766.5,0.737617,247.865005,49,54,333,19,46.356819,43.059174,2.102176,0.370421
3,3.07,0.184,66.98,5.8,7.6,7.5,1601,1787.0,0.895915,203.480229,44,59,395,21,55.633369,38.548523,15.127802,0.721031
4,3.17,0.187,68.74,7.2,8.0,7.1,1919,2372.0,0.809022,263.421354,62,58,454,19,60.012882,48.875011,55.774792,0.580292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,57.96,3.010,726.46,18.5,18.5,13.0,19084,22974.0,0.830678,767.938160,193,167,1367,33,190.580887,137.733093,62.138748,0.691160
161,81.46,3.880,1001.79,21.0,21.5,14.8,18373,21556.5,0.852318,661.612260,164,189,1317,29,176.040924,141.141724,22.056726,0.597653
162,140.84,6.380,1707.14,23.0,22.5,16.9,23374,26050.0,0.897274,683.754395,186,194,1427,31,191.379974,164.496170,175.959305,0.511091
163,108.17,5.250,1364.18,31.4,20.5,16.6,23457,25678.5,0.913488,671.754395,198,179,1364,31,194.815384,159.870819,122.656914,0.571464


In [113]:
LFW_train, LFW_test = data_workflow(DATA, 'LFW_g')

training data: <torch.utils.data.dataloader.DataLoader object at 0x7fdac7ffcd90>
 test data: <torch.utils.data.dataloader.DataLoader object at 0x7fdac7ffebc0>


In [114]:
class DeepNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(17, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        ) 

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits.double()
        



In [115]:
model = DeepNeuralNetwork()
model = model.double()
print(model)

DeepNeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=17, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [116]:
def train_loop(dataset, model, loss_function, optimizer):
    model.train()
    for (X, y) in dataset:
        #X, y = X.to('cuda'), y.to('cuda')
        y = y.view(-1,1) 
     
        prediction = model(X)
        loss = loss_function(prediction, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss}")
        #print(f"X: {X} \n Y: {y}")

In [117]:
def test_loop(dataset, model, loss_function):
    size = len(dataset.dataset)
    num_batches = len(dataset)

    test_loss, correct = 0, 0

    with torch.no_grad():
        for (X, y) in dataset:
            pred = model(X)
            test_loss += loss_function(pred, y).item()
            

    test_loss /= num_batches
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [118]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

for t in range(EPOCHS):
    print(f"Epoch {t+1}\n------------------------------------")
    train_loop(LFW_train, model, LOSS_FUNCTION, optimizer)
    #test_loop(LFW_test, model, LOSS_FUNCTION)
print("Training complete!")

Epoch 1
------------------------------------
Loss: 21431.507663081014
Loss: 20867.801295106703
Loss: 414.907548456538
Loss: 2442.380409976909
Loss: 14503.978838034003
Epoch 2
------------------------------------
Loss: 14812.625250216584
Loss: 9116.58982569032
Loss: 6589.808533840103
Loss: 1768.0930445530437
Loss: 1340.4782562482276
Epoch 3
------------------------------------
Loss: 1340.495341372728
Loss: 625.1939823663355
Loss: 4356.157994203428
Loss: 3374.583083790389
Loss: 1393.5825982882589
Epoch 4
------------------------------------
Loss: 4887.575492418273
Loss: 1496.1760495634426
Loss: 1297.0717938082964
Loss: 230.29227044742285
Loss: 128.53984202697828
Epoch 5
------------------------------------
Loss: 1242.1423665165833
Loss: 1237.2508189267992
Loss: 2277.8222465284343
Loss: 2273.6894372379998
Loss: 1115.5020031667113
Epoch 6
------------------------------------
Loss: 591.4608471849976
Loss: 379.55043875059204
Loss: 552.8405228447305
Loss: 407.87288802926975
Loss: 1029.5156483